<a href="https://colab.research.google.com/github/Tarun-619/AI-lab-1WA23CS015/blob/main/AI_lab7_015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resolution

In [2]:
def is_variable(x):
    return isinstance(x, str) and x[0].islower()

def substitute(theta, clause):
    return [(p, [theta.get(t, t) for t in terms]) for (p, terms) in clause]

def unify(x, y, theta):
    if theta is None: return None
    if x == y: return theta
    if is_variable(x): return unify_var(x, y, theta)
    if is_variable(y): return unify_var(y, x, theta)
    if isinstance(x, list) and isinstance(y, list) and len(x) == len(y):
        for xi, yi in zip(x, y):
            theta = unify(xi, yi, theta)
            if theta is None: return None
        return theta
    return None

def unify_var(v, x, theta):
    if v in theta: return unify(theta[v], x, theta)
    if x in theta: return unify(v, theta[x], theta)
    new = theta.copy()
    new[v] = x
    return new

def resolve(ci, cj):
    resolvents = []
    for pi in ci:
        for pj in cj:
            if pi[0].lstrip("¬") == pj[0].lstrip("¬") and (pi[0].startswith("¬") != pj[0].startswith("¬")):
                theta = unify(pi[1], pj[1], {})
                if theta is not None:
                    new_clause = substitute(theta, [x for x in ci if x != pi]) \
                               + substitute(theta, [x for x in cj if x != pj])
                    res = []
                    for lit in new_clause:
                        if lit not in res:
                            res.append(lit)
                    resolvents.append(res)
    return resolvents

def resolution(kb, query):
    clauses = [list(c) for c in kb] + [[("¬" + query[0], query[1])]]
    new = []

    while True:
        for i in range(len(clauses)):
            for j in range(i + 1, len(clauses)):
                for r in resolve(clauses[i], clauses[j]):
                    if r == []:
                        return True
                    if r not in new and r not in clauses:
                        new.append(r)

        if not new:
            return False

        clauses.extend(new)
        new = []

KB = [
    [("¬Food", ["x"]), ("Likes", ["Tarun", "x"])],
    [("Food", ["Apple"])],
    [("Food", ["Vegetable"])],
    [("¬Eats", ["a", "b"]), ("¬NotKilled", ["a"]), ("Food", ["b"])],
    [("Eats", ["TR", "Peanuts"])],
    [("Alive", ["TR"])],
    [("¬Eats", ["TR", "x"]), ("Eats", ["Harry", "x"])],
    [("¬Alive", ["x"]), ("NotKilled", ["x"])],
    [("¬NotKilled", ["x"]), ("Alive", ["x"])]
]

query = ("Likes", ["Tarun", "Peanuts"])
print("Proved?", resolution(KB, query))


Proved? True
